# Data Wrangling with Spark SQL Quiz

This quiz uses the same dataset and most of the same questions from the earlier "Quiz - Data Wrangling with Data Frames Jupyter Notebook." For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [1]:
from pyspark.sql import SparkSession
# from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
# from pyspark.sql.functions import sum as Fsum
# from pyspark.sql.window import Window
# from pyspark.sql.types import IntegerType

In [2]:
spark = SparkSession.builder.appName("Spark Sql Quiz").getOrCreate()

userlog = spark.read.json("data/sparkify_log_small.json")
userlog.createOrReplaceTempView("logtable")

# Question 1

Which page did user id ""(empty string) NOT visit?

In [3]:
userlog.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [6]:
spark.sql("SELECT * \
            FROM ( \
                SELECT DISTINCT page \
                FROM logtable \
                WHERE userID='') AS userpages \
            RIGHT JOIN ( \
                SELECT DISTINCT page \
                FROM logtable) AS allpages \
            ON userpages.page = allpages.page \
            WHERE userpages.page IS NULL").show()

+----+----------------+
|page|            page|
+----+----------------+
|null|Submit Downgrade|
|null|       Downgrade|
|null|          Logout|
|null|   Save Settings|
|null|        Settings|
|null|        NextSong|
|null|         Upgrade|
|null|           Error|
|null|  Submit Upgrade|
+----+----------------+



# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

#### You could choose both Spark Sql or Spark DataFrames. 

#### Sometimes you might prefer Spark Sql by syntax more clear and especially for teams that are experienced in SQL use. In other hand Spark Dataframes allows you to have much more control at the data you are manipulatting.

# Question 3

How many female users do we have in the data set?

In [7]:
spark.sql("SELECT COUNT(DISTINCT userID) \
            FROM logtable \
            WHERE gender = 'F'").show()

+----------------------+
|count(DISTINCT userID)|
+----------------------+
|                   462|
+----------------------+



# Question 4

How many songs were played from the most played artist?

In [10]:
spark.sql("SELECT Artist, COUNT(Artist) AS plays \
            FROM logtable \
            GROUP BY Artist \
            ORDER BY plays DESC \
            LIMIT 1").show()

+--------+-----+
|  Artist|plays|
+--------+-----+
|Coldplay|   83|
+--------+-----+



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [14]:
home = spark.sql("SELECT userID, page, ts, CASE WHEN page = 'Home' THEN 1 ELSE 0 END AS home FROM logtable \
                WHERE (page = 'NextSong') or (page = 'Home')")

home.createOrReplaceTempView("hometable")

cumsum = spark.sql("SELECT *, SUM(home) OVER \
                (PARTITION BY userID ORDER BY ts DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS period \
                   FROM hometable")

cumsum.createOrReplaceTempView("periodtable")

spark.sql("SELECT AVG(countresults) FROM \
            (SELECT COUNT(*) AS countresults FROM periodtable \
            GROUP BY userID, period, page HAVING page = 'NextSong') AS counts").show()


+-----------------+
|avg(countresults)|
+-----------------+
|6.898347107438017|
+-----------------+

